In [3]:
%env CUDA_VISIBLE_DEVICES=2
from transformers import  LlamaConfig, LlamaForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./tknz/tiny-ko-tokenizer", trust_remote_code=True)
print(len(tokenizer))

config = LlamaConfig(
    # --- 모델 구조 및 크기 관련 핵심 파라미터 ---
    
    hidden_size=256,        # [필수] 모델의 모든 레이어에서 사용되는 벡터의 차원(크기)입니다. 
                            # 임베딩, 어텐션 출력 등 모델의 기본 표현력을 결정하는 가장 중요한 파라미터 중 하나입니다.
                            # 값이 클수록 모델의 용량(capacity)이 커져 더 복잡한 정보를 학습할 수 있지만, 메모리 사용량과 연산량이 크게 증가합니다.
                            # (참고: Llama3-8B는 4096, TinyLlama는 2048을 사용합니다.)

    num_hidden_layers=12,    # [필수] 모델에 쌓을 트랜스포머 블록(레이어)의 총 개수입니다.
                            # 모델의 '깊이'를 결정하며, 깊을수록 더 추상적이고 복잡한 패턴을 학습할 수 있습니다.
                            # hidden_size와 함께 모델의 전체 파라미터 수를 결정하는 핵심 요소입니다.
                            # (참고: Llama3-8B는 32, TinyLlama는 22를 사용합니다.)
    
    intermediate_size=1024,  # [필수] 각 트랜스포머 블록 내부의 피드포워드 신경망(FFN)의 중간 레이어 크기입니다.
                            # 어텐션 메커니즘이 처리한 정보를 확장했다가 다시 축소하는 역할을 하여 모델의 학습 능력을 높입니다.
                            # 보통 hidden_size의 2.5배 ~ 4배 사이로 설정하며, (1536 / 576 ≈ 2.67배로 적절한 범위)
                            # 최근 모델들은 이 비율을 더 높이는 경향이 있습니다. (예: Llama3 ≈ 3.5배)
                            # hidden_size는 이 값으로 나누어져야 합니다. (hidden_size % num_attention_heads == 0)


    # --- etc, ---

    tie_word_embeddings=True,

    # --- 어텐션 메커니즘 관련 파라미터 ---

    num_attention_heads=4,   # [필수] 멀티 헤드 어텐션(Multi-Head Attention)에서 사용할 '헤드'의 개수입니다.
                            # 하나의 어텐션을 여러 개로 나누어 각각 다른 관점에서 정보의 연관성을 보도록 하는 효과가 있습니다.
                            # `hidden_size`는 반드시 `num_attention_heads`로 나누어떨어져야 합니다. (576 / 9 = 64)
                            # 이 결과값(64)이 각 어텐션 헤드의 차원(head_dim)이 됩니다.
                            # IMPORTANT: self.hidden_size // self.num_attention_heads = head_dim,
                            # head_dim should be one of: [ 256, 192, 128, 96, 80, 64 ]

    num_key_value_heads=2,   # [선택적, 성능향상용] Grouped-Query Attention (GQA)을 위한 파라미터입니다.
                            # 추론 시 속도 향상을 위해 여러 개의 쿼리 헤드(Q)가 하나의 키(K)/밸류(V) 헤드를 공유하도록 합니다.
                            # - `num_key_value_heads` == `num_attention_heads` : 일반적인 Multi-Head Attention (MHA)
                            # - `num_key_value_heads` == 1 : Multi-Query Attention (MQA)
                            # - 1 < `num_key_value_heads` < `num_attention_heads` : Grouped-Query Attention (GQA)
                            # 여기서는 9개의 쿼리 헤드가 2개의 키/밸류 헤드를 공유하며(2개씩 1그룹), 추론 시 메모리 대역폭을 절약하여 속도를 높입니다.
    
    # --- 토크나이저 및 입출력 관련 파라미터 ---
    vocab_size=len(tokenizer),
    max_position_embeddings=8192,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

print(f"Config: {config}")

# 랜덤 파라미터로 초기화
model = LlamaForCausalLM(config)

# 모델 파라미터 수 확인
model_size = sum(t.numel() for t in model.parameters())
print(f"Model size: {model_size/1000**3:.2f}B parameters")
print(f"Model size: {model_size/1000**2:.2f}M parameters")
print(f"Model size: {model_size/1000:.1f}K parameters")

model.save_pretrained("./tiny-random")
tokenizer.save_pretrained("./tiny-random")

model.push_to_hub("minpeter/tiny-ko-random", private=True)
tokenizer.push_to_hub("minpeter/tiny-ko-random", private=True)

env: CUDA_VISIBLE_DEVICES=2


AttributeError: partially initialized module 'torch' has no attribute 'types' (most likely due to a circular import)

In [7]:
import transformers
import torch

# 모델 ID와 파이프라인 준비는 동일합니다.
model_id = "./tiny-random" 

# device_map="auto"는 모델이 알아서 GPU/CPU에 할당되도록 합니다.
# 모델이 작으므로 device="cuda:0"으로 명시해도 좋습니다.
pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="auto",
)

prompt = "한글은 대한민국의"

# --- 1. 랜덤 샘플링을 활성화하여 생성하기 (do_sample=True) ---
print("--- 1. 랜덤 샘플링 (do_sample=True) 결과 ---")
print("실행할 때마다 결과가 달라질 수 있습니다.\n")

# do_sample=True로 샘플링을 켭니다.
# temperature: 창의성 조절 (1에 가까울수록 랜덤, 0에 가까울수록 정해진 답)
# top_p: 확률이 높은 순으로 단어 후보를 정함 (보통 0.9 ~ 0.95)
# max_new_tokens: 최대 생성 토큰 수
sampling_output = pipeline(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    max_new_tokens=100  
)[0]["generated_text"]

print(sampling_output)


# --- 2. 랜덤 샘플링을 비활성화하여 생성하기 (Beam Search) ---
print("\n\n--- 2. 결정론적 탐색 (Beam Search) 결과 ---")
print("실행할 때마다 항상 동일한 결과가 나옵니다.\n")

# num_beams > 1로 설정하면 Beam Search가 활성화됩니다.
# Beam Search는 do_sample=False를 자동으로 가정합니다.
# 여러 후보(beam)를 동시에 탐색하여 가장 확률 높은 문장을 찾으므로 품질이 좋습니다.
deterministic_output = pipeline(
    prompt,
    num_beams=6,
    max_new_tokens=100
)[0]["generated_text"]

print(deterministic_output)

Device set to use cuda:0


--- 1. 랜덤 샘플링 (do_sample=True) 결과 ---
실행할 때마다 결과가 달라질 수 있습니다.

한글은 대한민국의 미드필더 발달을선 이재용주희 틀 박쥐와닉샵 선지년대와바이 체험 심각 구조는 워크샵 수준과 선지 냈킷 이민 내 거대쏘 수준과 디카프킷/또는 공주 의성 수준과 뮬러 틀스트림 수준과 거대 이민 정산종 공주 쓰레기스트림 치료법을어는 바꾸어야 냈/또는 파트타임쏘 감염인들을쏘년과 이야기는 감염인들을정제 마셔 기다릴해질 이별을 책임 마셔킷 뮬러정제eld 카운 구멍을 책임 납부명령 아티스트넷째 약물과의 이별을 공주 수준과 이별을 심각 캐릭터를해질정제 치료법을정부가 심각 책임 이민 구멍을 워크샵 구멍을 심각 납부명령 모임에서 거대 배설물에는정부가 돌아오는 식품에계시록 기다릴 기다릴 심각보코


--- 2. 결정론적 탐색 (Beam Search) 결과 ---
실행할 때마다 항상 동일한 결과가 나옵니다.

한글은 대한민국의 구조는 구조는 구조는닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉닉
